# Exploratory Data Analysis

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import seaborn as sns
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.preprocessing import StandardScaler

In [4]:
df = pd.read_csv('../data/train.csv')
df.head()

,Id,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,...,Screen Porch,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,SalePrice
0,109,533352170,60,RL,NaN,13517,Pave,NaN,IR1,Lvl,...,0,0,NaN,NaN,NaN,0,3,2010,WD,130500
1,544,531379050,60,RL,43.0,11492,Pave,NaN,IR1,Lvl,...,0,0,NaN,NaN,NaN,0,4,2009,WD,220000
2,153,535304180,20,RL,68.0,7922,Pave,NaN,Reg,Lvl,...,0,0,NaN,NaN,NaN,0,1,2010,WD,109000
3,318,916386060,60,RL,73.0,9802,Pave,NaN,Reg,Lvl,...,0,0,NaN,NaN,NaN,0,4,2010,WD,174000
4,255,906425045,50,RL,82.0,14235,Pave,NaN,IR1,Lvl,...,0,0,NaN,NaN,NaN,0,3,2010,WD,138500


In [11]:
# check for nulls
print(df.shape)
print()

null_counts = df.isnull().sum()

null_columns_with_counts = {}

for key,value in null_counts.items():
    if value > 0:
        null_columns_with_counts[key] = {"count": value, "type": str(df[key].dtype)}

null_columns_with_counts

(2051, 81)



{'Lot Frontage': {'count': 330, 'type': 'float64'},
 'Alley': {'count': 1911, 'type': 'object'},
 'Mas Vnr Type': {'count': 22, 'type': 'object'},
 'Mas Vnr Area': {'count': 22, 'type': 'float64'},
 'Bsmt Qual': {'count': 55, 'type': 'object'},
 'Bsmt Cond': {'count': 55, 'type': 'object'},
 'Bsmt Exposure': {'count': 58, 'type': 'object'},
 'BsmtFin Type 1': {'count': 55, 'type': 'object'},
 'BsmtFin SF 1': {'count': 1, 'type': 'float64'},
 'BsmtFin Type 2': {'count': 56, 'type': 'object'},
 'BsmtFin SF 2': {'count': 1, 'type': 'float64'},
 'Bsmt Unf SF': {'count': 1, 'type': 'float64'},
 'Total Bsmt SF': {'count': 1, 'type': 'float64'},
 'Bsmt Full Bath': {'count': 2, 'type': 'float64'},
 'Bsmt Half Bath': {'count': 2, 'type': 'float64'},
 'Fireplace Qu': {'count': 1000, 'type': 'object'},
 'Garage Type': {'count': 113, 'type': 'object'},
 'Garage Yr Blt': {'count': 114, 'type': 'float64'},
 'Garage Finish': {'count': 114, 'type': 'object'},
 'Garage Cars': {'count': 1, 'type': 'floa

I created a function to set default values based on what I had seen in the CSV file for empty or "NA" values.

In [59]:
def clean_data(data_frame):
    # loop through columns with null data
    for column in ['Lot Frontage','Alley','Mas Vnr Type','Mas Vnr Area','Bsmt Qual','Bsmt Cond','Bsmt Exposure','BsmtFin Type 1','BsmtFin SF 1','BsmtFin Type 2','BsmtFin SF 2','Bsmt Unf SF','Bsmt Unf SF','Total Bsmt SF','Bsmt Full Bath','Bsmt Half Bath','Fireplace Qu','Garage Type','Garage Yr Blt','Garage Finish','Garage Cars','Garage Area','Garage Qual','Garage Cond','Fence','Misc Feature', 'Pool QC']:
        if column in ['Lot Frontage','Mas Vnr Area','BsmtFin SF 1','BsmtFin SF 2','Bsmt Unf SF','Bsmt Unf SF','Total Bsmt SF','Bsmt Full Bath','Bsmt Half Bath','Garage Cars', 'Garage Area']:
            data_frame[column].fillna(0, inplace=True)
        elif column in ['Alley','Mas Vnr Type','Bsmt Qual','Bsmt Cond','Bsmt Exposure','BsmtFin Type 1','BsmtFin Type 2','Fireplace Qu','Garage Type','Garage Finish','Garage Qual','Garage Cond','Fence','Misc Feature','Pool QC']:
            data_frame[column].fillna("NA", inplace=True)
        elif column == 'Garage Yr Blt':
            data_frame[column].fillna("NA", inplace=True)
            data_frame[column] = data_frame[column].map(lambda x: str(x))

clean_data(df)

In [74]:
def add_up_square_feet(data_frame):
    # Set an initital value
    data_frame["total_square_feet"] = 0

    # Add up the values from the other columns minus the total column
    for col in [col for col in data_frame.columns if " SF" in col]:
        data_frame["total_square_feet"] += data_frame[col]
        

# retrieve the first 5 columns
add_up_square_feet(df)


df["total_square_feet"].head()

0    2973.0
1    4022.0
2    3223.0
3    2312.0
4    2856.0
Name: total_square_feet, dtype: float64

In [91]:
X = df[['Year Built','total_square_feet','Overall Qual']]
y = df['SalePrice']

X_train,X_test, y_train, y_test = train_test_split(X, y)
ss = StandardScaler()
X_train_scaled = ss.fit_transform(X_train)
X_test_scaled = ss.transform(X_test)
rf = RandomForestRegressor()
rf.fit(X_train, y_train)
print(rf.score(X_test, y_test))
rf.fit(X_train_scaled, y_train)
print(rf.score(X_test_scaled, y_test))
print(mean_squared_error(y_test, rf.predict(X_test)))
print(mean_squared_error(y_test, rf.predict(X_test_scaled)))


print(rf.feature_importances_)


[2.63885296e-03 3.79842368e-03 1.93100983e-03 3.17441107e-03
 5.64193117e-03 2.47657619e-01 6.00223387e-03 6.21996902e-02
 8.18172428e-03 3.85976299e-03 5.82643710e-03 7.55112879e-05
 5.31504038e-03 3.77759996e-03 4.90897391e-03 4.92274561e-03
 4.86607952e-05 2.25332228e-02 5.18426900e-04 7.38855365e-05
 1.30257955e-03 1.30888851e-03 5.75032250e-04 1.25945210e-05
 2.63353432e-03 1.33088369e-03 8.51299799e-03 8.84998070e-03
 1.34283358e-03 2.76545395e-03 6.48917184e-04 1.35521895e-04
 3.44361067e-03 1.52335332e-05 1.86407492e-05 2.56087809e-03
 8.23071392e-04 5.02460641e-01 0.00000000e+00 7.58108954e-05
 8.68591856e-05 0.00000000e+00 1.27581548e-06 5.00036897e-04
 7.84686805e-04 9.81194103e-08 0.00000000e+00 2.90648091e-05
 9.01061201e-05 2.56530585e-05 2.79075716e-04 1.87531405e-04
 0.00000000e+00 7.05048064e-04 6.08980830e-05 4.86717573e-04
 1.56015305e-05 5.97914022e-04 0.00000000e+00 0.00000000e+00
 0.00000000e+00 1.19030677e-04 2.57532643e-04 1.91061271e-05
 3.58078064e-06 2.846139

In [82]:

lr.fit(X_train, y_train)
print(lr.score(X_test, y_test))
lr.fit(X_train_scaled, y_train)
print(lr.score(X_test_scaled, y_test))
print(mean_squared_error(y_test, lr.predict(X_test)))
print(mean_squared_error(y_test, lr.predict(X_test_scaled)))

0.7497779516663003
0.7497779516663003
2.4115664735904412e+16
1595042588.2050862


In [84]:
rf = RandomForestRegressor()
rf.fit(X_train, y_train)
print(rf.score(X_test, y_test))
rf.fit(X_train_scaled, y_train)
print(rf.score(X_test_scaled, y_test))
print(mean_squared_error(y_test, rf.predict(X_test)))
print(mean_squared_error(y_test, rf.predict(X_test_scaled)))


0.8036268652125279
0.8126021387522205
26912236825.331635
1194569270.0514085
[0.10288377 0.40087945 0.49623678]


In [71]:
clf = Lasso(alpha=.05,normalize=True, max_iter=1e5)
clf.fit(X_train_scaled,y_train)
clf.score(X_test_scaled,y_test)
mean_squared_error(y_test, clf.predict(X_test_scaled))

1595032543.7364116

In [72]:
rr = Ridge(alpha=1,max_iter=1e5)
rr.fit(X_train_scaled,y_train)
rr.score(X_test_scaled,y_test)
mean_squared_error(y_test, rr.predict(X_test_scaled))

1594898565.4947398

In [75]:
en = ElasticNet(alpha=.1,max_iter=1e5)
en.fit(X_train_scaled,y_train)
en.score(X_test_scaled,y_test)
mean_squared_error(y_test, en.predict(X_test_scaled))

1588285653.3254771

In [76]:
# Plot the charts to see how far off you are

Finally, I fitted the kaggle test data, created a file, and uploaded the submission via the kaggle CLI.

In [78]:
# https://www.kaggle.com/dansbecker/submitting-from-a-kernel
# Read the test data
test = pd.read_csv('../data/test.csv')
clean_data(test)
add_up_square_feet(test)

# Treat the test data in the same way as training data. In this case, pull same columns.
test_X = test[['Year Built','total_square_feet','Overall Qual']]
scaled_test_X = ss.transform(test_X)

# # Use the model to make predictions
predicted_prices = en.predict(scaled_test_X)
my_submission = pd.DataFrame({'Id': test.Id, 'SalePrice': predicted_prices})

file_name = '../submissions/kaggle.csv'
my_submission.to_csv(file_name, index=False)

In [79]:
import subprocess, webbrowser
result = subprocess.check_output(f'kaggle competitions submit -f {file_name} -m "uploading a new set" dsi-us-5-project-2-regression-challenge')
if result == b'Successfully submitted to DSI-US-5 Project 2 Regression Challenge':
    webbrowser.open("https://www.kaggle.com/c/dsi-us-5-project-2-regression-challenge/leaderboard")
else:
    print(result)